In [68]:
import numpy as np
np.set_printoptions(precision=15)

In [69]:
label = ['Party', 'Pub', 'TV', 'study']
predictor = ['Tired', 'Hungover', 'Scared', 'Fine']

In [70]:
trans_matrix = np.array([
    [0.05, 0.05, 0.7, 0.2],
    [0.1, 0.05, 0.6, 0.25],
    [0.1, 0.3, 0.4, 0.2],
    [0.25, 0.4, 0.3, 0.05]
])

trans_matrix.sum(axis=1) # If all equals 1, it is valid

array([1., 1., 1., 1.])

In [71]:
emis_matrix = np.array([
    [0.3, 0.4, 0.2, 0.1],
    [0.4, 0.2, 0.1, 0.3],
    [0.2, 0.1, 0.2, 0.5],
    [0.3, 0.05, 0.3, 0.35]
])

emis_matrix.sum(axis=1) # If all equals 1, it is valid

array([1., 1., 1., 1.])

In [72]:
y0 = np.array([0.25, 0.25, 0.25, 0.25])

In [77]:
target = ['Tired', 'Tired', 'Scared']

def check_valid_target(target, label):
    return [t in label for t in target]

check_valid_target(target, label)

[False, False, False]

In [74]:
def forward_algorithm(label, predictor, trans, emis, y0, target):
    # Initialize result matrix
    ret = np.zeros((len(label), len(target)))
    # Calculate y1
    print("We have:")
    print("alpha_0(y0) = p(x0 | y0) * p(y0) = p(x0, y0)")
    
    print("Then")
    x_idx = predictor.index(target[0])
    for i, val in enumerate(label):
        print("alpha_0({0}) = p(x0 = {1} | y0 = {0}) * p(y0 = {0}) = {2} * {3} = {4}".format(val, target[0], emis[i, x_idx], y0[i], emis[i, x_idx] * y0[i]))
        ret[i, 0] = emis[i, x_idx] * y0[i]
    
    print()
    for i in range(1, len(target)):
        print("alpha_1(y{1}) = p(x{1} | y{1}) * sum_(y{0}) (alpha_{0}(y{0}) * p(y{1} | y{0}))".format(i-1, i))
        x_idx = predictor.index(target[i])
        for j, val in enumerate(label):
            print("alpha_{0}({1}) = {2} *".format(i, val, emis[j, x_idx]))
            total = 0
            for k in range(len(label)):  
                print("{0} * {1} {2}".format(ret[k, i-1], trans[k, j], '+' if k != len(label) - 1 else ''))
                total += ret[k, i-1] * trans[k, j]
            total *= emis[j, x_idx]
            print("= {0}".format(np.round(total, 8)))
            ret[j, i] = np.round(total, 8)
    return ret
        
forward_algorithm(label, predictor, trans_matrix, emis_matrix, y0, target)

We have:
alpha_0(y0) = p(x0 | y0) * p(y0) = p(x0, y0)
Then
alpha_0(Party) = p(x0 = Tired | y0 = Party) * p(y0 = Party) = 0.3 * 0.25 = 0.075
alpha_0(Pub) = p(x0 = Tired | y0 = Pub) * p(y0 = Pub) = 0.4 * 0.25 = 0.1
alpha_0(TV) = p(x0 = Tired | y0 = TV) * p(y0 = TV) = 0.2 * 0.25 = 0.05
alpha_0(study) = p(x0 = Tired | y0 = study) * p(y0 = study) = 0.3 * 0.25 = 0.075

alpha_1(y1) = p(x1 | y1) * sum_(y0) (alpha_0(y0) * p(y1 | y0))
alpha_1(Party) = 0.3 *
0.075 * 0.05 +
0.1 * 0.1 +
0.05 * 0.1 +
0.075 * 0.25 
= 0.01125
alpha_1(Pub) = 0.4 *
0.075 * 0.05 +
0.1 * 0.05 +
0.05 * 0.3 +
0.075 * 0.4 
= 0.0215
alpha_1(TV) = 0.2 *
0.075 * 0.7 +
0.1 * 0.6 +
0.05 * 0.4 +
0.075 * 0.3 
= 0.031
alpha_1(study) = 0.3 *
0.075 * 0.2 +
0.1 * 0.25 +
0.05 * 0.2 +
0.075 * 0.05 
= 0.016125
alpha_1(y2) = p(x2 | y2) * sum_(y1) (alpha_1(y1) * p(y2 | y1))
alpha_2(Party) = 0.1 *
0.01125 * 0.05 +
0.0215 * 0.1 +
0.031 * 0.1 +
0.016125 * 0.25 
= 0.00098438
alpha_2(Pub) = 0.3 *
0.01125 * 0.05 +
0.0215 * 0.05 +
0.031 * 0.3 +
0.

array([[0.075     , 0.01125   , 0.00098438],
       [0.1       , 0.0215    , 0.00521625],
       [0.05      , 0.031     , 0.01900625],
       [0.075     , 0.016125  , 0.00512094]])

In [81]:
def viterbi_algorithm(label, predictor, trans, emis, y0, target):
    # Initialize result matrix
    ret = np.zeros((len(label), len(target)))
    cal = np.zeros((len(label) ** 2, len(target)))
    # Calculate y1
    print("We have:")
    print("p(x1, y1) = p(x1 | y1) * p(y1)")
    
    print("Then")
    x_idx = predictor.index(target[0])
    for i, val in enumerate(label):
        print("p(y1 = {0}) = p(x1 = {1} | y1 = {0}) * p(y1 = {0}) = {2} * {3} = {4}".format(val, target[0], emis[i, x_idx], y0[i], emis[i, x_idx] * y0[i]))
        ret[i, 0] = emis[i, x_idx] * y0[i]
    
    for i in range(1, len(target)):
        print()
        print("p(x{0}, x{1}, y{0}, y{1}) = p(y{1} | x{1}) * p(y{1} | y{0}) * p(x{0}, y{0})".format(i, i+1))
        for j, val in enumerate(label):
            for k, val2 in enumerate(label):
#                 print("p({0}, {1}, {2}, {3}) = p({3} | {1}) * p({3} | {2}) * p({0}, {2}) = {4} * {5} * {6} = {7}".format(
                print("p(x{8} = {0}, x{9} = {1}, y{8} = {2}, y{9} = {3}) = {4} * {5} * {6} = {7}".format(
                    target[i-1],
                    target[i],
                    val,
                    val2,
                    emis[j, predictor.index(target[i])],
                    trans[k, j],
                    ret[k, i-1],
                    np.round(emis[j, predictor.index(target[i])] * trans[k, j] * ret[k, i-1], 8),
                    i,
                    i+1
                ))
                cal[k + len(label) * j, i] = np.round(emis[j, predictor.index(target[i])] * trans[k, j] * ret[k, i-1], 8)
            print("max(.) = {0}".format(np.max(cal[len(label) * j:len(label) * (j+1)][:, i])))
            ret[j, i] = np.max(cal[len(label) * j:len(label) * (j+1)][:, i])
        
    return cal.astype(str), ret
        
viterbi_algorithm(label, predictor, trans_matrix, emis_matrix, y0, target)

We have:
p(x1, y1) = p(x1 | y1) * p(y1)
Then
p(y1 = Party) = p(x1 = Tired | y1 = Party) * p(y1 = Party) = 0.3 * 0.25 = 0.075
p(y1 = Pub) = p(x1 = Tired | y1 = Pub) * p(y1 = Pub) = 0.4 * 0.25 = 0.1
p(y1 = TV) = p(x1 = Tired | y1 = TV) * p(y1 = TV) = 0.2 * 0.25 = 0.05
p(y1 = study) = p(x1 = Tired | y1 = study) * p(y1 = study) = 0.3 * 0.25 = 0.075

p(x1, x2, y1, y2) = p(y2 | x2) * p(y2 | y1) * p(x1, y1)
p(x1 = Tired, x2 = Tired, y1 = Party, y2 = Party) = 0.3 * 0.05 * 0.075 = 0.001125
p(x1 = Tired, x2 = Tired, y1 = Party, y2 = Pub) = 0.3 * 0.1 * 0.1 = 0.003
p(x1 = Tired, x2 = Tired, y1 = Party, y2 = TV) = 0.3 * 0.1 * 0.05 = 0.0015
p(x1 = Tired, x2 = Tired, y1 = Party, y2 = study) = 0.3 * 0.25 * 0.075 = 0.005625
max(.) = 0.005625
p(x1 = Tired, x2 = Tired, y1 = Pub, y2 = Party) = 0.4 * 0.05 * 0.075 = 0.0015
p(x1 = Tired, x2 = Tired, y1 = Pub, y2 = Pub) = 0.4 * 0.05 * 0.1 = 0.002
p(x1 = Tired, x2 = Tired, y1 = Pub, y2 = TV) = 0.4 * 0.3 * 0.05 = 0.006
p(x1 = Tired, x2 = Tired, y1 = Pub, y2 = s

(array([['0.0', '0.001125', '5.625e-05'],
        ['0.0', '0.003', '0.00024'],
        ['0.0', '0.0015', '0.00024'],
        ['0.0', '0.005625', '0.000375'],
        ['0.0', '0.0015', '2.813e-05'],
        ['0.0', '0.002', '6e-05'],
        ['0.0', '0.006', '0.00036'],
        ['0.0', '0.012', '0.0003'],
        ['0.0', '0.0105', '0.0007875'],
        ['0.0', '0.012', '0.00144'],
        ['0.0', '0.004', '0.00096'],
        ['0.0', '0.0045', '0.00045'],
        ['0.0', '0.0045', '0.0003375'],
        ['0.0', '0.0075', '0.0009'],
        ['0.0', '0.003', '0.00072'],
        ['0.0', '0.001125', '0.0001125']], dtype='<U32'),
 array([[0.075   , 0.005625, 0.000375],
        [0.1     , 0.012   , 0.00036 ],
        [0.05    , 0.012   , 0.00144 ],
        [0.075   , 0.0075  , 0.0009  ]]))